In [43]:
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import matplotlib.pyplot as plt
import pandas as pd
import io
import os
import re
import string
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (
  Embedding,Input,LSTM,Input,Conv2D,Softmax,Dropout,Dense,GRU,
  MaxPooling2D,LayerNormalization,Reshape,BatchNormalization,Bidirectional)
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

In [44]:
print(tf.__version__)

2.13.1


In [45]:
transcripts = pd.read_csv('Dataset/data.csv')

transcripts.head()

,path,sentence
0,common_voice_id_39599471.mp3,dia tidak sepenuhnya mempercayaiku
1,common_voice_id_32165882.mp3,aku juga tidak mengerti
2,common_voice_id_27657418.mp3,berharap kau di sini
3,common_voice_id_23033570.mp3,maha suci allah
4,common_voice_id_40088845.mp3,pada akhirnya dia memilih anak kucing yang sat...


In [51]:
# Path audio
# audio_file = "Dataset/clips/common_voice_id_39599471.mp3"

def load_audio():
    # Membaca file MP3
    audio_binary = tf.io.read_file("Dataset/clips/common_voice_id_39599471.mp3")
    
    # Mendekode file MP3 menjadi tensor audio
    audio_tensor = tfio.audio.decode_mp3(audio_binary)

    return audio_tensor

print(load_audio())

2024-11-22 04:28:00.664046: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: Dataset/clips/common_voice_id_39599471.mp3; No such file or directory


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dataset/clips/common_voice_id_39599471.mp3; No such file or directory [Op:ReadFile]

In [30]:
file_path = "Dataset/clips/common_voice_id_39599471.mp3"
waveform, sample_rate = load_mp3(file_path)

In [31]:
# Fungsi untuk normalisasi waveform
def normalize_waveform(waveform):
    # Cari amplitudo maksimum
    max_amplitude = tf.reduce_max(tf.abs(waveform))
    
    # Normalisasi waveform ke rentang [-1, 1]
    normalized_waveform = waveform / max_amplitude
    return normalized_waveform

In [32]:
# menambahkan padding jika perlu
def pad_waveform(waveform, frame_length, frame_step):
    # Hitung jumlah padding yang diperlukan di dimensi waktu
    waveform_length = tf.shape(waveform)[0]
    padding_needed = frame_length - (waveform_length % frame_step)
    
    # Jika padding diperlukan, tambahkan padding di dimensi waktu
    if padding_needed != 0:
        padded_waveform = tf.pad(waveform, paddings=[[0, padding_needed]])
    else:
        padded_waveform = waveform
    
    return padded_waveform

In [33]:
# Normalisasi
normalized_waveform = normalize_waveform(waveform)

# Cetak informasi
print(f"Sample rate: {sample_rate}")
print(f"Waveform shape sebelum di normalisasi: {waveform.shape}")
print(f"Waveform shape setelah di normalisasi: {normalized_waveform.shape}")

Sample rate: 16000
Waveform shape sebelum di normalisasi: (47229, 1)
Waveform shape setelah di normalisasi: (47229, 1)


In [36]:
# spektrogram dari waveform
def create_spectrogram(waveform, frame_length=256, frame_step=128, fft_length=256):
    # Hitung STFT
    stft = tf.signal.stft(
        waveform,
        frame_length=frame_length,
        frame_step=frame_step,
        fft_length=fft_length
    )
    
    # Ambil magnitudo (hilangkan fase)
    spectrogram = tf.abs(stft)
    return spectrogram

frame_length = 256  # Panjang frame untuk STFT
frame_step = 128    # Langkah antar frame

# spektrogram dari waveform yang sudah dinormalisasi
spectrogram = create_spectrogram(normalized_waveform)

print(f"Spectrogram shape: {spectrogram.shape}")

Spectrogram shape: (47229, 0, 129)


In [3]:
import os
import pandas as pd

# Path folder tempat file audio berada
folder_path = "Dataset/clips"

# Load data_wav.csv
csv_path = "Dataset/filtered_data_wav.csv"
df = pd.read_csv(csv_path)

# Ambil nama file dari kolom 'path'
file_names_to_keep = set(df['path'])

# Loop melalui semua file di folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    # Jika file tidak ada dalam daftar, hapus
    if file_name not in file_names_to_keep:
        os.remove(file_path)
        print(f"{file_name} telah dihapus.")
    else:
        print(f"{file_name} dipertahankan.")

common_voice_id_19987659.wav dipertahankan.
common_voice_id_20996390.wav dipertahankan.
common_voice_id_19804955.wav telah dihapus.
common_voice_id_26258168.wav dipertahankan.
common_voice_id_19270624.wav dipertahankan.
common_voice_id_35864204.wav dipertahankan.
common_voice_id_26254934.wav dipertahankan.
common_voice_id_26270126.wav dipertahankan.
common_voice_id_20957301.wav telah dihapus.
common_voice_id_20971925.wav telah dihapus.
common_voice_id_20977307.wav dipertahankan.
common_voice_id_22185214.wav dipertahankan.
common_voice_id_19761316.wav telah dihapus.
common_voice_id_22918167.wav dipertahankan.
common_voice_id_35278828.wav dipertahankan.
common_voice_id_39922661.wav dipertahankan.
common_voice_id_23821260.wav dipertahankan.
common_voice_id_19648516.wav dipertahankan.
common_voice_id_25129285.wav telah dihapus.
common_voice_id_27853719.wav telah dihapus.
common_voice_id_20991158.wav dipertahankan.
common_voice_id_25129167.wav dipertahankan.
common_voice_id_34731696.wav dip